In [9]:
import pandas as pd
import sqlite3
import re


csv_file = 'lecture_plan_v1.csv'


db_file = 'kwu-lecture-database-v7.db'


df = pd.read_csv(csv_file)


def extract_class_rooms(lecture_times):
    if pd.isna(lecture_times):
        return ""
    rooms = re.findall(r'\((.*?)\)', lecture_times)
    if rooms == ["미지정"]:
        return ""
    unique_rooms = list(set(rooms))
    if '미지정' in unique_rooms:
        unique_rooms.remove('미지정')
    return ','.join(unique_rooms)


df['lecClassRoom'] = df['강의시간 및 강의실'].apply(extract_class_rooms)


conn = sqlite3.connect(db_file)
cursor = conn.cursor()


for index, row in df.iterrows():
    lec_number = row['학정번호']
    lec_class_room = row['lecClassRoom']
    cursor.execute("""
        UPDATE LectureTable
        SET lecClassRoom = ?
        WHERE lecNumber = ?
    """, (lec_class_room, lec_number))


conn.commit()
conn.close()

In [14]:
import sqlite3
import re


conn = sqlite3.connect('kwu-lecture-database-v8.db')
cursor = conn.cursor()


def replace_string(text):
    replacements = {
        '연': '연구',
        '비': '비마',
        '참': '참빛',
        '옥': '옥의',
        '기': '기념',
        '화': '화도',
        '새': '새빛',
        '누': '누리'
    }

    for key, value in replacements.items():
        pattern = fr'{key}(?=[0-9a-zA-Z])'
        text = re.sub(pattern, value, text)

    return text


cursor.execute("SELECT lecNumber, lecClassRoom FROM LectureTable")
rows = cursor.fetchall()


for row in rows:
    lecNumber, lecClassRoom = row
    new_lecClassRoom = replace_string(lecClassRoom)
    if new_lecClassRoom != lecClassRoom:
        cursor.execute(
            "UPDATE LectureTable SET lecClassRoom = ? WHERE lecNumber = ?", (new_lecClassRoom, lecNumber))


conn.commit()


conn.close()